In [2]:
import requests
import pandas as pd
import numpy as np
import pickle

In [3]:
def save_pkl(file_dir, data):
    f = open(file_dir,"wb")
    pickle.dump(data, f, protocol=4)
    f.close()
    
def read_pkl(file_dir):
    f = open(file_dir,"rb")
    data = pickle.load(f)
    return data

In [4]:
df = pd.read_csv("Aal/GT.csv")
df = df.rename(columns={"Recorded":"time","heat_index":"temperature","rh":"humidity","wdir":"wind_direction","wspd":"wind_speed"})
df = df[["station_id","time","temperature","pressure","humidity","wind_direction","wind_speed"]]
dfGade = df.iloc[:39462,:]
dfTag = df.iloc[39462:,:]
dfTag["station_id"] = "Aal_grid_001"
dfGade["station_id"] = "Aal_grid_000"
dfTag = dfTag.iloc[:4034,:]
dfGade= dfGade.iloc[:4034,:]
df = pd.concat([dfGade,dfTag])
df

C:\Users\Nobody\AppData\Local\Temp/ipykernel_8704/1545880212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTag["station_id"] = "Aal_grid_001"
C:\Users\Nobody\AppData\Local\Temp/ipykernel_8704/1545880212.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGade["station_id"] = "Aal_grid_000"


,station_id,time,temperature,pressure,humidity,wind_direction,wind_speed
0,Aal_grid_000,2019-12-06 10:00:00,9.0,992.65,87.0,260.0,31.0
1,Aal_grid_000,2019-12-06 10:30:00,9.0,993.65,87.0,260.0,33.0
2,Aal_grid_000,2019-12-06 11:00:00,8.0,993.65,93.0,250.0,33.0
3,Aal_grid_000,2019-12-06 11:30:00,8.0,993.65,93.0,250.0,30.0
4,Aal_grid_000,2019-12-06 12:00:00,8.0,993.65,93.0,250.0,24.0
...,...,...,...,...,...,...,...
43491,Aal_grid_001,2020-02-28 08:30:00,1.0,1000.64,93.0,260.0,11.0
43492,Aal_grid_001,2020-02-28 09:00:00,1.0,1001.64,100.0,260.0,11.0
43493,Aal_grid_001,2020-02-28 09:30:00,1.0,1001.64,100.0,270.0,7.0
43494,Aal_grid_001,2020-02-28 10:00:00,1.0,1001.64,100.0,270.0,11.0


In [5]:
# df.rename(columns={'utc_time': 'time',
#                    'wind_speed/kph': 'wind_speed'
#                   }, inplace=True)
df['time'] = pd.to_datetime(df['time'])
df.index = df['time']
df = df.sort_index()
group = df.groupby("station_id")
meo_group = {}
for name, g in group:
    meo_group[name] = g.sort_index()
save_pkl("Aal/meo_train.pkl", meo_group)
df.tail(3)

,station_id,time,temperature,pressure,humidity,wind_direction,wind_speed
time,,,,,,,
2020-02-28 10:00:00,Aal_grid_000,2020-02-28 10:00:00,1.0,1001.64,100.0,270.0,11.0
2020-02-28 10:30:00,Aal_grid_000,2020-02-28 10:30:00,2.0,1002.64,100.0,260.0,11.0
2020-02-28 10:30:00,Aal_grid_001,2020-02-28 10:30:00,2.0,1002.64,100.0,260.0,11.0


In [6]:
meo_group = read_pkl("Aal/meo_train.pkl")

In [7]:
# nearst = {
#     'GB0': 'london_grid_451', 'CD1': 'london_grid_388', 'HV1': 'london_grid_472', 'CD9': 'london_grid_409',
#     'TD5': 'london_grid_366', 'GR4': 'london_grid_451', 'RB7': 'london_grid_452', 'TH4': 'london_grid_430',
#     'HR1': 'london_grid_368', 'BL0': 'london_grid_409', 'GR9': 'london_grid_430', 'ST5': 'london_grid_408',
#     'LH0': 'london_grid_346', 'KF1': 'london_grid_388', 'MY7': 'london_grid_388', 'LW2': 'london_grid_430',
#     'GN3': 'london_grid_451', 'GN0': 'london_grid_451', 'CT3': 'london_grid_409', 'CR8': 'london_grid_408',
#     'BX1': 'london_grid_472' # ,'CT2': 'london_grid_409'
# }

nearst = {
    "Gade":"Aal_grid_000","Tag":"Aal_grid_001"
}

In [8]:
# read the original features (original version of station_group) from air_processing.pkl

station_group = read_pkl("Aal/air_processing.pkl")
# for name in station_group:
#     print(station_group[name])
station_position = read_pkl("Aal/station_processing.pkl")
# station_position

In [9]:
# merge the weather condition features
attr_need = ["temperature", "pressure", "humidity", "wind_direction", "wind_speed"]
for name in station_group:
    # print(name)
    
    l = len(station_group[name])
    print(l)
    grid_id = nearst[name]
    for attr in attr_need:
        keep_values = meo_group[grid_id][attr].values
        print(len(keep_values))
        extra_array = np.zeros(l - len(keep_values))
        station_group[name][attr] = np.append(keep_values, extra_array)
#     print(station_group[name])
station_group['Gade']

10897
4034
4034
4034
4034
4034
10897
4034
4034
4034
4034
4034


,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed
time,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,87.0,260.0,31.0
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,87.0,260.0,33.0
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,33.0
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,30.0
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,93.0,250.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,0.0,0.0,0.0


In [10]:
# merge the holiday condition feature
df_h = pd.read_csv("Aal/holiday_aal2.csv")
holiday_dict = {}
date_list = df_h['date'].values
holiday_list = df_h['holiday'].values
for i in range(len(date_list)):
    holiday_dict[str(date_list[i])] = holiday_list[i]
for name in station_group:
    # print(name)
    time_list = station_group[name].index.values
    temp_list = []
    for time in time_list:
        time = str(time)
        date = time[:4]+time[5:7]+time[8:10]
        temp_list.append(holiday_dict[date])
    station_group[name]['holiday'] = temp_list
#     print(station_group[name])
station_group['Gade']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday
time,,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,87.0,260.0,31.0,0
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,87.0,260.0,33.0,0
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,33.0,0
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,30.0,0
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,93.0,250.0,24.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,0.0,0.0,0.0,0


In [11]:
# merge the feature of station index

NO2_mean_list = []

for name in station_group:
    NO2_mean = np.mean(station_group[name]['NO2_Concentration'].values)
    NO2_mean_list.append(NO2_mean)
        
for name in station_group:
    small_time = 0
    large_time = 0
    NO2_mean = np.mean(station_group[name]['NO2_Concentration'].values)
    for i in range(len(NO2_mean_list)):
        if NO2_mean_list[i] > NO2_mean:
            small_time = small_time + 1
        else:
            large_time = large_time + 1
            
    l = len(station_group[name])        
    station_group[name]['station_index'] = np.full((l), small_time)
    
station_group['Tag']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday,station_index
time,,,,,,,,,,,,,,,
2019-12-06 10:00:00,Tag,6.43,8.06,2019,12,4,6,10,9.0,992.65,87.0,260.0,31.0,0,1
2019-12-06 10:30:00,Tag,9.36,11.62,2019,12,4,6,10,9.0,993.65,87.0,260.0,33.0,0,1
2019-12-06 11:00:00,Tag,10.42,13.22,2019,12,4,6,11,8.0,993.65,93.0,250.0,33.0,0,1
2019-12-06 11:30:00,Tag,9.28,12.08,2019,12,4,6,11,8.0,993.65,93.0,250.0,30.0,0,1
2019-12-06 12:00:00,Tag,13.09,17.45,2019,12,4,6,12,8.0,993.65,93.0,250.0,24.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Tag,4.85,6.60,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,1
2020-07-20 08:30:00,Tag,4.84,6.58,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,1
2020-07-20 09:00:00,Tag,4.82,6.57,2020,7,0,20,9,0.0,0.00,0.0,0.0,0.0,0,1


In [12]:
# merge the feature of pressure/temperature，temperature/humidity，PM25+PM10，PM25/PM10

for name in station_group:
    df = station_group[name]
    df['pressure_temperature'] = df.apply(lambda x: x['pressure'] * x['temperature'], axis=1)
    df['temperature_humidity'] = df.apply(lambda x: x['temperature'] / (x['humidity']+0.01), axis=1)
    df['NO2*NOx'] = df.apply(lambda x: x['NO2_Concentration'] + x['NOx_Concentration'], axis=1)  
    df['NO2_NOx'] = df.apply(lambda x: x['NO2_Concentration'] / (x['NOx_Concentration']+0.01), axis=1)
    
station_group['Gade']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday,station_index,pressure_temperature,temperature_humidity,NO2*NOx,NO2_NOx
time,,,,,,,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,87.0,260.0,31.0,0,0,8933.85,0.103436,30.07,0.649123
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,87.0,260.0,33.0,0,0,8942.85,0.103436,43.38,0.662452
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,33.0,0,0,7949.20,0.086012,53.20,0.672744
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,30.0,0,0,7949.20,0.086012,44.83,0.556404
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,93.0,250.0,24.0,0,0,7949.20,0.086012,63.78,0.649173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,21.66,0.504861
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,26.17,0.515046
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,28.48,0.486176


In [13]:
df = station_group['Gade']
group = df.groupby("time_hour")
hour_group = {}
for hour, g in group:
    hour_group[str(hour)] = g.sort_index()
for hour in hour_group:
    hour_df = hour_group[hour]
    print(hour_df)

                    station_id  NO2_Concentration  NOx_Concentration  \
time                                                                   
2019-12-07 00:00:00       Gade               8.43              10.23   
2019-12-07 00:30:00       Gade               9.08              10.67   
2019-12-08 00:00:00       Gade              23.34              34.53   
2019-12-08 00:30:00       Gade              19.56              41.76   
2019-12-09 00:00:00       Gade               3.61               3.42   
...                        ...                ...                ...   
2020-07-18 00:30:00       Gade               6.26              10.38   
2020-07-19 00:00:00       Gade              12.14              16.89   
2020-07-19 00:30:00       Gade              12.16              16.99   
2020-07-20 00:00:00       Gade               2.62               3.63   
2020-07-20 00:30:00       Gade               2.30               2.58   

                     time_year  time_month  time_week  time_day

In [14]:

for name in station_group:
    df = station_group[name]
    NO2_list = df['NO2_Concentration'].values
    NOx_list = df['NOx_Concentration'].values
    temperature_list = df['temperature'].values
    pressure_list = df['pressure'].values
    humidity_list = df['humidity'].values
    wind_speed_list = df['wind_speed'].values
    wind_direction_list = df['wind_direction'].values

    hour_number = 14 * 24 #what is this number

    mean_NOx_list = []
    mean_NO2_list = []
    mean_temperature_list = []
    mean_pressure_list = []
    mean_humidity_list = []
    mean_wind_speed_list = []
    mean_wind_direction_list = []
  
    max_NOx_list = []
    max_NO2_list = []
    max_temperature_list = []
    max_pressure_list = []
    max_humidity_list = []
    max_wind_speed_list = []
    
    var_NOx_list = []
    var_NO2_list = []
    var_temperature_list = []
    var_pressure_list = []
    var_humidity_list = []
    var_wind_speed_list = []
    var_wind_direction_list = []
    for i in range(hour_number-1):
        mean_NOx_list.append(0)
        mean_NO2_list.append(0)
        mean_temperature_list.append(0)
        mean_pressure_list.append(0)
        mean_humidity_list.append(0)
        mean_wind_speed_list.append(0)
        mean_wind_direction_list.append(0)
        max_NOx_list.append(0)
        max_NO2_list.append(0)
        max_temperature_list.append(0)
        max_pressure_list.append(0)
        max_humidity_list.append(0)
        max_wind_speed_list.append(0)
        var_NOx_list.append(0)
        var_NO2_list.append(0)
        var_temperature_list.append(0)
        var_pressure_list.append(0)
        var_humidity_list.append(0)
        var_wind_speed_list.append(0)
        var_wind_direction_list.append(0)

    for i in range(hour_number-1, len(NO2_list)):
        NOx_array = NOx_list[i-hour_number+1:i+1]
        NO2_array = NO2_list[i-hour_number+1:i+1]
        temperature_array = temperature_list[i-hour_number+1:i+1]
        pressure_array = pressure_list[i-hour_number+1:i+1]
        humidity_array = humidity_list[i-hour_number+1:i+1]
        wind_speed_array = wind_speed_list[i-hour_number+1:i+1]
        wind_direction_array = wind_direction_list[i-hour_number+1:i+1]


        mean_NOx_list.append(np.mean(NOx_array))
        mean_NO2_list.append(np.mean(NO2_array))
        mean_temperature_list.append(np.mean(temperature_array))
        mean_pressure_list.append(np.mean(pressure_array))
        mean_humidity_list.append(np.mean(humidity_array))
        mean_wind_speed_list.append(np.mean(wind_speed_array))
        mean_wind_direction_list.append(np.mean(wind_direction_array))
        max_NOx_list.append(np.max(NOx_array))
        max_NO2_list.append(np.max(NO2_array))
        max_temperature_list.append(np.max(temperature_array))
        max_pressure_list.append(np.max(pressure_array))
        max_humidity_list.append(np.max(humidity_array))
        max_wind_speed_list.append(np.max(wind_speed_array))
        var_NOx_list.append(np.var(NOx_array))
        var_NO2_list.append(np.var(NO2_array))
        var_temperature_list.append(np.var(temperature_array))
        var_pressure_list.append(np.var(pressure_array))
        var_humidity_list.append(np.var(humidity_array))
        var_wind_speed_list.append(np.var(wind_speed_array))
        var_wind_direction_list.append(np.var(wind_direction_array))
    #     print(np.mean(array))
    #     print(np.max(array))
    #     print(np.var(array))


    df['mean_NO2'] = mean_NO2_list
    df['mean_NOx'] = mean_NOx_list
    df['mean_temperature'] = mean_temperature_list
    df['mean_pressure'] = mean_pressure_list
    df['mean_humidity'] = mean_humidity_list
    df['mean_wind_speed'] = mean_wind_speed_list
    df['mean_wind_direction'] = mean_wind_direction_list
    df['max_NO2'] = max_NO2_list
    df['max_NOx'] = max_NOx_list
    df['max_temperature'] = max_temperature_list
    df['max_pressure'] = max_pressure_list
    df['max_humidity'] = max_humidity_list
    df['max_wind_speed'] = max_wind_speed_list
    df['var_NO2'] = var_NO2_list
    df['var_NOx'] = var_NOx_list
    df['var_temperature'] = var_temperature_list
    df['var_pressure'] = var_pressure_list
    df['var_humidity'] = var_humidity_list
    df['var_wind_speed'] = var_wind_speed_list
    df['var_wind_direction'] = var_wind_direction_list

In [15]:
station_group['Gade']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,...,max_pressure,max_humidity,max_wind_speed,var_NO2,var_NOx,var_temperature,var_pressure,var_humidity,var_wind_speed,var_wind_direction
time,,,,,,,,,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,...,0.0,0.0,0.0,68.182141,357.905090,0.0,0.0,0.0,0.0,0.0
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,...,0.0,0.0,0.0,68.131640,357.766441,0.0,0.0,0.0,0.0,0.0
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,...,0.0,0.0,0.0,68.122623,357.671515,0.0,0.0,0.0,0.0,0.0


In [16]:
def show(key):
    s = ['Gade','Tag']
    for name in station_group:
        if name in s:
            df = station_group[name]
            temp = df[key].values
            t = []
            for i in range(len(temp)-1):
                t.append(abs(temp[i]-temp[i+1]))
    print(np.mean(t))
show('NO2_Concentration')
show('NOx_Concentration')
show('temperature')
show('pressure')
show('humidity')
show('wind_speed')
show('wind_direction')

1.1463160792951541
1.4985058737151247
0.09113436123348018
0.20743116740088124
0.6966776798825257
0.9141886930983847
3.526064610866373


In [17]:
# save all of the features (final version of station_group) into station_final.pkl
save_pkl("Aal/station_final.pkl", station_group)

In [18]:
# load the file
station_group = read_pkl("aal/station_final.pkl")
station_group['Gade']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,...,max_pressure,max_humidity,max_wind_speed,var_NO2,var_NOx,var_temperature,var_pressure,var_humidity,var_wind_speed,var_wind_direction
time,,,,,,,,,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,...,0.0,0.0,0.0,68.182141,357.905090,0.0,0.0,0.0,0.0,0.0
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,...,0.0,0.0,0.0,68.131640,357.766441,0.0,0.0,0.0,0.0,0.0
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,...,0.0,0.0,0.0,68.122623,357.671515,0.0,0.0,0.0,0.0,0.0
